In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer

/tmp/ipykernel_34557/3934139722.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Easy

In [2]:
df = pd.read_csv('../data/audi.csv')
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [3]:
df = df.drop(columns=['transmission', 'fuelType', 'model']) # работаем только с числовыми признаками
df.head()

,year,price,mileage,tax,mpg,engineSize
0,2017,12500,15735,150,55.4,1.4
1,2016,16500,36203,20,64.2,2.0
2,2016,11000,29946,30,55.4,1.4
3,2017,16800,25952,145,67.3,2.0
4,2019,17300,1998,145,49.6,1.0


In [4]:
X = df.drop(columns=['price'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X.head()

,year,mileage,tax,mpg,engineSize
0,2017,15735,150,55.4,1.4
1,2016,36203,20,64.2,2.0
2,2016,29946,30,55.4,1.4
3,2017,25952,145,67.3,2.0
4,2019,1998,145,49.6,1.0


In [6]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(r2_score(y_pred, y_test))

0.9086067964789499


In [7]:
model.feature_importances_

array([0.25763737, 0.04983374, 0.03931078, 0.46154097, 0.19167714])

Наиболее значительный признак - это `mpg`. `year` также немного влияет на результат, остальные незначительны.

# Medium

In [8]:
df_audi = pd.read_csv('../data/audi.csv')
df_bmw = pd.read_csv('../data/bmw.csv')
df_cclass = pd.read_csv('../data/cclass.csv')
df_focus = pd.read_csv('../data/focus.csv')
df_ford = pd.read_csv('../data/ford.csv')
df_hyundi = pd.read_csv('../data/hyundi.csv')
df_merc = pd.read_csv('../data/merc.csv')
df_skoda = pd.read_csv('../data/skoda.csv')
df_toyota = pd.read_csv('../data/toyota.csv')
df_vauxhall = pd.read_csv('../data/vauxhall.csv')
df_vw = pd.read_csv('../data/vw.csv')

In [9]:
df = pd.concat([df_audi, df_bmw, df_cclass, df_focus, df_ford, df_hyundi, df_merc, 
                df_skoda, df_toyota, df_vauxhall, df_vw], ignore_index=True)

In [10]:
df.shape

(108540, 10)

In [11]:
print(set(df['transmission']))
print(set(df['fuelType']))
print(set(df['model']))

{'Other', 'Automatic', 'Semi-Auto', 'Manual'}
{'Hybrid', 'Diesel', 'Petrol', 'Other', 'Electric'}
{' M5', ' Z3', ' Zafira Tourer', ' Transit Tourneo', ' IQ', ' Auris', ' 6 Series', ' Amica', '180', ' Caddy', ' Puma', ' RS6', ' Q8', ' S5', ' Veloster', ' Sharan', ' Ampera', ' M6', ' M4', ' Escort', ' G Class', ' R Class', ' Avensis', ' A7', ' Touran', ' Scala', ' I20', ' Grandland X', ' 2 Series', ' Getz', ' 1 Series', ' Santa Fe', ' X6', ' CLS Class', ' Superb', ' Camry', ' Up', ' GTC', ' RS4', ' Verso', ' Fabia', ' GLA Class', ' Corsa', ' Mustang', ' S-MAX', ' Z4', ' A Class', ' I800', ' Mokka X', ' B-MAX', ' RS7', ' I10', ' Insignia', ' Passat', ' Golf', ' T-Roc', ' Golf SV', ' M Class', ' Eos', ' R8', ' Fusion', ' C Class', ' Citigo', ' S3', ' I30', ' Karoq', ' California', ' Q3', ' C-HR', ' GLS Class', ' Tigra', ' Tucson', ' RAV4', ' Crossland X', ' A2', ' Viva', ' M3', ' SQ5', ' Tourneo Custom', ' Caddy Maxi Life', ' Terracan', ' Mondeo', ' Urban Cruiser', ' Streetka', ' Ioniq', '

`transmission`, `fuelType` и `model` заменим на последовательные числа от 0 до n-1, где n - количество различных значений признака.

In [12]:
features_to_change = {'transmission' : list(set(df['transmission'])),
            'fuelType' : list(set(df['fuelType'])),
            'model' : list(set(df['model']))}

def change_feature(df, feature, feature_list):
    for i in df.index:
        df.loc[i, feature] = feature_list.index(df.loc[i, feature])
    return df

In [13]:
for feature in features_to_change.keys():
    df = change_feature(df, feature, features_to_change[feature])
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,92,2017,12500,3,15735,2,150.0,55.4,1.4,NaN
1,131,2016,16500,1,36203,1,20.0,64.2,2.0,NaN
2,92,2016,11000,3,29946,2,30.0,55.4,1.4,NaN
3,118,2017,16800,1,25952,1,145.0,67.3,2.0,NaN
4,192,2019,17300,3,1998,2,145.0,49.6,1.0,NaN


In [14]:
params_decision_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 20], 
    'max_features': ['sqrt', 'log2', None]
}

params_random_forest = {
    'n_estimators': [5, 50, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 20] #log2(10^5) is close to 16
}

# Возьмем маленькую подвыборку данных в силу ограниченности вычислительных ресурсов

X = df.drop(columns=['price'])
y = df['price']
X, _, y, _ = train_test_split(X, y, train_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X.shape

(21708, 9)

In [16]:
%%time
search1 = RandomizedSearchCV(DecisionTreeClassifier(), params_decision_tree, scoring=make_scorer(r2_score))
search1.fit(X_train, y_train)
search1.best_estimator_

/home/nixiiee/miniconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


CPU times: user 29.8 s, sys: 1.84 s, total: 31.6 s
Wall time: 31.6 s


DecisionTreeClassifier(max_depth=10)

In [17]:
y_pred1 = search1.predict(X_test)
r2_score(y_pred1, y_test)

-1.40150715856665

In [18]:
X = df.drop(columns=['price'])
y = df['price']
X, _, y, _ = train_test_split(X, y, train_size=0.02, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
%%time
search2 = GridSearchCV(RandomForestClassifier(), params_random_forest, scoring=make_scorer(r2_score))
search2.fit(X_train, y_train)
search2.best_estimator_

/home/nixiiee/miniconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


CPU times: user 39.6 s, sys: 1.74 s, total: 41.3 s
Wall time: 41.4 s


RandomForestClassifier(max_depth=20)

In [20]:
y_pred2 = search2.predict(X_test)
r2_score(y_pred2, y_test)

0.630411477666643

RandomForest победил, несмотря на то, что обучался на меньшей выборке

# Hard

Я скачала архив телеграм-канала с объявлениями о продаже машин. Будем парсить оттуда сообщения про ауди.

In [21]:
import json
with open('../data/result.json') as f:
    data = json.load(f)

In [22]:
messages = []
for obj in data['messages']:
    if len(obj['text']) > 0 and type(obj['text']) != type("") and {'type': 'hashtag', 'text': '#Audi'} in obj['text']:
        messages.append(obj['text'])

In [23]:
messages[:3]

[['В рамках услуги «Подбор под ключ» в Москве был осмотрен автомобиль ',
  {'type': 'hashtag', 'text': '#Audi'},
  ' ',
  {'type': 'hashtag', 'text': '#A3'},
  '\n⠀\n-—\n1️⃣ Год выпуска: 2015\n2️⃣ Пробег: 99 679 км.\n3️⃣ Двигатель: 1.4 л. / 125 л.с. / бензин\n4️⃣ КПП: робот\n5️⃣ Привод: передний\n6️⃣ Владельцы: 2\n-—\n✅ Два владельца ( фактически эксплуатировал один владелец-девушка) ПТС оригинал\n✅ Кузов без серьёзных кузовных ремонтов\n✅ Хороший внешний вид\n✅ Осмотр на подъемнике без нареканий\n✅ Подтверждённый пробег\n✅ Редкая комплектация: адаптивный ксеноновый свет, круиз-контроль, спорт-салон, 3-х спицевый скошенный руль, датчик света-дождя, противотуманные фары, расширенный пакет освещения (диодный), зеркало заднего вида (безрамочное), мультимедиа MMI, drive select, Bluetooth, оригинальный видео-регистратор\n✅ Отличная цена\n-—\n🚫 Эксплуатационные дефекты\n🚫 Отсутствует зимний комплект резины\n-—\nЦена: 1 086 000 рублей\n-—\n',
  {'type': 'hashtag', 'text': '#автоподбор_рекомен

In [24]:
len(messages)

141

In [25]:
df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize', 'tax(£)'],
      dtype='object')

Про mpg в датасете нет информации

In [26]:
def get_tax(n):
    if n <= 100:
        return 2.5 * n
    elif n > 100 and n <= 150:
        return 3.5 * n
    elif n > 150 and n <= 200:
        return 5 * n
    elif n > 200 and n <= 250:
        return 7.5 * n
    else:
        return 15 * n

In [27]:
t = messages[0][4].split('\n')
t

['',
 '⠀',
 '-—',
 '1️⃣ Год выпуска: 2015',
 '2️⃣ Пробег: 99 679 км.',
 '3️⃣ Двигатель: 1.4 л. / 125 л.с. / бензин',
 '4️⃣ КПП: робот',
 '5️⃣ Привод: передний',
 '6️⃣ Владельцы: 2',
 '-—',
 '✅ Два владельца ( фактически эксплуатировал один владелец-девушка) ПТС оригинал',
 '✅ Кузов без серьёзных кузовных ремонтов',
 '✅ Хороший внешний вид',
 '✅ Осмотр на подъемнике без нареканий',
 '✅ Подтверждённый пробег',
 '✅ Редкая комплектация: адаптивный ксеноновый свет, круиз-контроль, спорт-салон, 3-х спицевый скошенный руль, датчик света-дождя, противотуманные фары, расширенный пакет освещения (диодный), зеркало заднего вида (безрамочное), мультимедиа MMI, drive select, Bluetooth, оригинальный видео-регистратор',
 '✅ Отличная цена',
 '-—',
 '🚫 Эксплуатационные дефекты',
 '🚫 Отсутствует зимний комплект резины',
 '-—',
 'Цена: 1 086 000 рублей',
 '-—',
 '']

In [28]:
messages[0][4].split('\n')[3][messages[0][4].split('\n')[3].find(':')+1:]

' 2015'

In [29]:
d = {'model' : [],
    'year' : [],
    'price' : [],
    'transmission' : [],
    'mileage' : [],
    'fuelType' : [],
    'tax' : [],
    'engineSize' : []}

prev_len = 0

for message in messages:
    for element in message:
        if isinstance(element, dict) and element['text'] not in ['#Audi', '#автоподбор_не_рекомендует', '#автоподбор_рекомендует'] and len(element['text']) == 3:
            d['model'].append(element['text'][1:])
        else:
            try:
                if '\n' in element:
                    to_parse = element.split('\n')
                    count = 0
                    for point in to_parse:
                        if '-' in point:
                            count += 1
                        if count >= 2 and not('Цена' in point):
                            continue
                        if 'Год выпуска' in point:
                            d['year'].append(int(point[point.find(':')+2:]))
                        elif 'Пробег' in point:
                            d['mileage'].append(int(point[point.find(':')+2:point.find('км')-1].replace(' ', '')))
                        elif 'Двигатель' in point and '/' in point:
                            temp = point.split('/')
                            d['fuelType'].append(temp[-1][1:])
                            t = temp[0]
                            d['engineSize'].append(float(t[t.find(':')+2:t.find('л.')-1]))
                            t = temp[1][1:temp[1].find('л.')]
                            d['tax'].append(get_tax(int(t)))
                        elif 'Цена' in point and count >= 4:
                            if 'торг' in point:
                                d['price'].append(int(point[point.find('торга')+6:point.find('руб')-1].replace(' ', '').replace(' ', '')))
                            else:
                                d['price'].append(int(point[point.find(':')+2:point.find('руб')-1].replace(' ', '').replace(' ', '')))
                        elif 'КПП' in point:
                             d['transmission'].append(point[point.find(':')+2:])
            except Exception as e: 
                for key in d.keys():
                    d[key].pop()
                print(e)
                print()
                print(element)
                print()
                break
    for key in d.keys():
        if len(d[key]) < prev_len + 1:
            for _ in range(prev_len + 1 - len(d[key])):
                d[key].append(None)
        elif len(d[key]) > prev_len + 1:
            for _ in range(prev_len + 1 - len(d[key])):
                d[key].pop()
    prev_len += 1

invalid literal for int() with base 10: 'бензин'


-
🔘Год выпуска: 2013
🔘Пробег: 98000
🔘Двигатель: 2.0 / бензин / 211 л.с.
🔘КПП: Робот 
🔘Привод: Полный  
🔘Владельцы: 6
-
Цена: 1200000 рублей 
-
✅Оригинальный пробег  
✅Юридически чист, без залогов и обременений 
✅Два комплекта резины 
-
🚫Автомобиль восстановлен после тотального ДТП, окрашен почти весь, нарушена силовая структура кузова 
🚫Отсутствуют несколько подушек безопасности 
🚫ПТС дубликат 
🚫Цепь ГРМ уже подходит к замене, так же шумит маховик
-


invalid literal for int() with base 10: 'бензин'


-
🔘Год выпуска: 2016
🔘Пробег: 69000 км
🔘Двигатель: 1.4 / бензин / 150 л.с.
🔘КПП:  Робот
🔘Привод: Передний 
🔘Владельцы: 1
-
✅Юридически чист, без залогов и обременений 
✅Силовая структура кузова целая, геометрия заводская 
✅Хорошее техническое состояние
✅Птс оригинал
✅Хорошая комплектация 
✅Оригинальный пробег, присутствует история обслуживания
- 
🚫Несколько окрасов по кузову 
-
Цена: 1600000 рублей 
-


invalid literal for int() with 

Часть сэмплов не обработалась, нестрашно

In [30]:
for key in d.keys():
    print(key, len(d[key]))

model 141
year 141
price 141
transmission 141
mileage 141
fuelType 141
tax 141
engineSize 141


In [31]:
my_df = pd.DataFrame(d)
my_df.head()

,model,year,price,transmission,mileage,fuelType,tax,engineSize
0,A3,2015.0,1086000.0,робот,99679.0,бензин,437.5,1.4
1,Q5,2017.0,3050000.0,робот,43000.0,бензин,1867.5,2.0
2,A5,2009.0,930000.0,автоматическая,115000.0,дизель,1792.5,3.0
3,Q7,2017.0,4000000.0,автоматическая,30000.0,дизель,1867.5,3.0
4,Q3,2013.0,1200000.0,робот,91000.0,бензин,850.0,2.0


In [32]:
my_df = my_df.dropna()
my_df.shape

(119, 8)

На практике мы узнали, что Catboost нормально работает с категориальными фичами. Обучим его.

In [33]:
from catboost import CatBoostRegressor

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score

x_train, x_test, y_train, y_test = train_test_split(my_df.drop('price', axis=1), my_df['price'])

model = CatBoostRegressor()
model.fit(x_train, y_train, cat_features=['transmission', 'model', 'fuelType'])

Learning rate set to 0.027937
0:	learn: 1463331.6671409	total: 50.4ms	remaining: 50.4s
1:	learn: 1442152.9296471	total: 54.5ms	remaining: 27.2s
2:	learn: 1423276.0549209	total: 56.4ms	remaining: 18.7s
3:	learn: 1405582.4321197	total: 58.6ms	remaining: 14.6s
4:	learn: 1386387.1599707	total: 61.8ms	remaining: 12.3s
5:	learn: 1368826.8805021	total: 64.9ms	remaining: 10.8s
6:	learn: 1359616.2853820	total: 67.9ms	remaining: 9.63s
7:	learn: 1344886.0770652	total: 71.2ms	remaining: 8.83s
8:	learn: 1324145.0777070	total: 72.4ms	remaining: 7.97s
9:	learn: 1305296.8896302	total: 74.9ms	remaining: 7.41s
10:	learn: 1289288.5290251	total: 77.4ms	remaining: 6.96s
11:	learn: 1273869.1108381	total: 79.8ms	remaining: 6.57s
12:	learn: 1261236.2279259	total: 81.6ms	remaining: 6.2s
13:	learn: 1249063.2935902	total: 83.6ms	remaining: 5.88s
14:	learn: 1234798.9667787	total: 85.2ms	remaining: 5.59s
15:	learn: 1216490.8389225	total: 86.2ms	remaining: 5.3s
16:	learn: 1197933.7188992	total: 87.6ms	remaining: 5.

In [35]:
y_pred = model.predict(x_test)
print(r2_score(y_pred, y_test))
print(model.feature_importances_)

0.8598094613230057
[ 7.38781112 45.24924702  3.93188283  3.34812501  8.58735121 19.22365798
 12.27192482]


Одной из самых важных фич по прежнему остается год